### Load NHTSA and Ford AsBuilt Data for vehicles

Filtered to only include the Escape for this example

In [1]:
from nhtsa import decode_vins
from vins import load_vins

vins = load_vins(filter_comment='escape')
df = decode_vins(vins)
print(df.to_string())

Loaded 255 VINs (filter_comment='escape', include_openpilot=False)
Found AsBuilt data for 255 VINs
          ABS             ActiveSafetySysNote AdaptiveCruiseControl AdaptiveDrivingBeam                  AirBagLocFront     AirBagLocKnee             AirBagLocSide AutoReverseSystem AutomaticPedestrianAlertingSound Axles BatteryKWh         BatteryType BlindSpotMon                                                BodyClass BrakeSystemType         BusType  CAN_AACN       CIB DaytimeRunningLight DisplacementCC   DisplacementCI DisplacementL Doors              DriveType DynamicBrakeSupport       EDR       ESC                           ElectrificationLevel EngineConfiguration EngineCylinders EngineHP EngineManufacturer                     EngineModel ForwardCollisionWarning                                FuelInjectionType FuelTypePrimary            FuelTypeSecondary                                           GVWR KeylessIgnition LaneCenteringAssistance LaneDepartureWarning LaneKeepSystem LowerBea

In [2]:
# Count unique values in ModelYear
print(df['ModelYear'].value_counts())

# Filter ModelYear
df = df[df.ModelYear.astype(int) == 2020]

ModelYear
2020    85
2021    53
2022    48
2023    33
2019    27
2024     6
2018     2
2015     1
Name: count, dtype: int64


### Reduce to interesting columns

The NHTSA data has lots of columns, so we can filter down to only include the ones which don't change or aren't interesting to us.

In [3]:
SKIP = [
  'AirBagLocSide',
  'DisplacementCC',
  'DisplacementCI',
  'EngineCylinders',
  'LowerBeamHeadlampLightSource',
  'WheelSizeFront',
  'WheelSizeRear',
  'VIN',
  'VehicleDescriptor',
]

KEEP = [
  'FuelTypeSecondary',
]

properties = {}

for col in df.columns:
  if col in SKIP:
    continue

  property_values = set(df[col].unique())
  if '' in property_values and col not in KEEP:
    continue
  if len(property_values) == 1:
    continue

  properties[col] = property_values

properties

{'DisplacementL': {'1.5', '2.0', '2.5'},
 'DriveType': {'4WD/4-Wheel Drive/4x4', '4x2'},
 'FuelTypeSecondary': {'', 'Electric'},
 'Series': {'S', 'SE', 'SE FHEV', 'SEL', 'Titanium', 'Titanium FHEV'}}

### Combine NHTSA and Ford AsBuilt Data

We fetch the factory part numbers (software and hardware) from the Ford AsBuilt data and combine it with the NHTSA data.

In [4]:
# import pandas as pd

from panda.python.uds import DATA_IDENTIFIER_TYPE

from asbuilt import AsBuiltData
from ecu import FordEcu
from settings import VehicleSetting, VehicleSettings


df_fw = df.copy()


def get_ecu_identifier(ecu: FordEcu, identifier: int):
  def apply(row):
    data = AsBuiltData.from_vin(row['VIN'])
    if ecu not in data.ecus:
      return ''
    return data.get_identifier(ecu, identifier)
  return apply


def get_setting(setting: VehicleSetting):
  def apply(row):
    data = AsBuiltData.from_vin(row['VIN'])
    if setting.ecu not in data.ecus:
      return ''
    return data.get_setting_value(setting)
  return apply


# Drop rows that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df.columns if col not in properties and col != 'VIN'],
  inplace=True,
)

# Add the ECU identifiers
ecus = {
  'abs': FordEcu.AntiLockBrakeSystem,
  # 'engine': FordEcu.PowertrainControlModule,
  # 'eps': FordEcu.PowerSteeringControlModule,
  'fwdCamera': FordEcu.ImageProcessingModuleA,
  'fwdRadar': FordEcu.CruiseControlModule,
}
for name, ecu in ecus.items():
  df_fw[f'{name}_fw'] = df_fw.apply(
    get_ecu_identifier(
      ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER
    ),
    axis=1,
  )
  df_fw[f'{name}_part'] = df_fw.apply(get_ecu_identifier(ecu, 0xF111), axis=1)

# Apply filters
constants = {
  # 'ModelYear': '2020',
  # 'Series': 'Escape',
  # 'DisplacementL': '1.5',
  # 'DriveType': '4x2',
  # 'EngineCylinders': '3',
  # 'Series': 'Titanium',
}
for col, value in constants.items():
  df_fw = df_fw[df_fw[col] == value]
  df_fw.drop(columns=[col], inplace=True)

# Drop columns that are all empty
df_fw = df_fw.loc[:, (df_fw != '').any(axis=0)]

# Add settings
settings = {
  # TODO: read this from multiple modules to check that it's consistent
  'acc': VehicleSettings.ipma_enable_adaptive_cruise,
  'lca': VehicleSettings.ipma_enable_traffic_jam_assist,
}
for name, setting in settings.items():
  df_fw[f'code_{name}'] = df_fw.apply(get_setting(setting), axis=1)

# Drop the VIN
df_fw.drop(columns=['VIN'], inplace=True)

# Drop columns that are all the same
# df_fw = df_fw.loc[:, df_fw.apply(pd.Series.nunique) != 1]

# Add asterisks to column names that contain only one value
df_fw.rename(
  columns={
    col: f'*{col}' if len(set(df_fw[col].unique())) == 1 else col
    for col in df_fw.columns
  },
  inplace=True,
)

# Sort by columns
df_fw.sort_values(
  by=[f'code_{name}' for name in settings.keys()] + ['Series', 'DriveType', 'DisplacementL'] + [f'{name}_fw' for name in ecus.keys()],
  ascending=False,
  inplace=True,
  ignore_index=True,
)

print(df_fw.to_string())

   DisplacementL              DriveType FuelTypeSecondary         Series         abs_fw        abs_part    fwdCamera_fw *fwdCamera_part     fwdRadar_fw   fwdRadar_part     code_acc code_lca
0            2.5                    4x2          Electric  Titanium FHEV  LX6C-2D053-SA  LX6C-14F065-MA  LJ6T-14F397-AD  LJ6T-14F403-CA  LB5T-14D049-AB  LB5T-14F089-AA  RadarFusion       On
1            2.5                    4x2          Electric  Titanium FHEV  LX6C-2D053-SA  LX6C-14F065-MA  LJ6T-14F397-AD  LJ6T-14F403-CA  LB5T-14D049-AB  LB5T-14F089-AA  RadarFusion       On
2            2.5                    4x2          Electric  Titanium FHEV  LX6C-2D053-SA  LX6C-14F065-MA  LJ6T-14F397-AD  LJ6T-14F403-CA  LB5T-14D049-AB  LB5T-14F089-AA  RadarFusion       On
3            2.5                    4x2          Electric  Titanium FHEV  LX6C-2D053-NT  LX6C-14F065-MA  LJ6T-14F397-AD  LJ6T-14F403-CA  LB5T-14D049-AB  LB5T-14F089-AA  RadarFusion       On
4            2.5  4WD/4-Wheel Drive/4x4          E